In [26]:
import json
import collections
from tqdm import tqdm
import numpy as np
import pandas as pd
import random

import paddle
from paddle.io import Dataset, DataLoader
from paddle import nn
import paddle.nn.functional as F

import os
import pickle
from tqdm import tqdm

In [27]:
if paddle.device.is_compiled_with_cuda():
    paddle.device.set_device('gpu')
else:
    paddle.device.set_device('cpu')
print(paddle.device.get_device())
def seed_paddle(seed):
    seed = int(seed)
   
    random.seed(seed) 

    os.environ['PYTHONHASHSEED'] = str(seed) 
    np.random.seed(seed)
    paddle.seed(seed) 
seed_paddle(seed=1024)

gpu:0


In [28]:
size = 11
seq_len = size * size

In [29]:
class TsDataset(Dataset):
    def __init__(self, df):

        self.seq_list1 = list(df['Wspd_seq'])
        self.seq_list2 = list(df['Patv_seq'])
        
        self.seq_list3 = list(df['Pab1_seq'])
        self.seq_list4 = list(df['Temp_seq'])
        
        self.seq_list5 = list(df['Patv_space'])

        
        self.label_list = df.target.values

    def __getitem__(self, index):
        
        seq = self.seq_list1[index][-seq_len:]+self.seq_list2[index][-seq_len:]+\
                  self.seq_list3[index][-seq_len:]+self.seq_list4[index][-seq_len:]
        seq = np.array(seq).astype('float') 
        seq.resize(size, size, 4)
        
        image = np.array(self.seq_list5[index]).astype('float') 
        image.resize(11, 11, 1)

        label = np.array( self.label_list[index] ).astype( 'float' )
        
        seq = paddle.to_tensor(seq)
        space_data = paddle.to_tensor(image)
        

        return seq, space_data, label


    def __len__(self):
        return len(self.seq_list1)

In [30]:
class CNN(nn.Layer):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.cnnLayer1 = nn.Sequential(
        nn.Conv2D(4, 64, kernel_size=3, stride=1, padding=2),
        nn.BatchNorm2D(64),
        nn.GELU(),
        nn.AdaptiveAvgPool2D((size, size))
        )
        self.res1 = nn.Conv2D(4, 64, kernel_size=1, stride=1)
          
        self.cnnLayer2 = nn.Sequential(
            nn.Conv2D(64, 128, kernel_size=3, stride=1, padding=2), 
            nn.BatchNorm2D(128),
            nn.GELU(),
            nn.AdaptiveAvgPool2D((3, 3))
        )
        
        
        self.cnnLayer3 = nn.Sequential(
        nn.Conv2D(1, 64, kernel_size=3, stride=1, padding=1), 
        nn.BatchNorm2D(64),
        nn.GELU(),
        nn.AdaptiveMaxPool2D((3,3)))
       
        self.Linear1 = nn.Linear(4, 8, bias_attr=True)
        
        self.Linear2 = nn.Linear(2, 4, bias_attr=True)
        self.Linear3 = nn.Linear(12, 1, bias_attr=True)
        #self.Linear4 = nn.Linear(4, 1, bias_attr=True)

        
    def forward(self, X, space_data):

        cnn_out1 = self.cnnLayer1(X)
        res_out1 = self.res1(X)
        output1 = cnn_out1 + res_out1
        
        output2 = self.cnnLayer2(output1)
        output2 = paddle.reshape(output2, ( output2.shape[0], 288, -1))
        out1 = self.Linear1(output2)
        
        outputsp = self.cnnLayer3(space_data)
        outputsp = paddle.reshape(outputsp, ( outputsp.shape[0], 288, -1))
        out2 = self.Linear2(outputsp)

        final_input = out1
        final_input = paddle.concat(x=[final_input, out2], axis=2)
        final_output = self.Linear3(final_input)
       
        return final_output

In [31]:
model = CNN()
optimizer = paddle.optimizer.Adam(learning_rate=0.0005, parameters=model.parameters())
criterion = nn.MSELoss()

In [32]:
df_train = pd.read_csv('../data/train_data2022.csv')

In [33]:
len(df_train)

44349

In [34]:
cols = [x for x in df_train.columns if 'seq' in x or x=='target' or x=='Patv_space']
for col in cols:
    df_train[col] = df_train[col].apply(lambda x: json.loads(x))  
def get_diff(row):
    diff = np.array(row['Etmp_seq'])-np.array(row['Itmp_seq'])
    return list(diff)

df_train['Temp_seq'] = df_train.apply(get_diff, axis=1)

In [35]:
train_dataset = TsDataset(df_train)

train_loader = DataLoader(train_dataset,
                      batch_size=128,
                      shuffle=True,
                      num_workers=4)

In [36]:
ls = []
for epoch in range(50):
    model.train()
    pred_list = []
    label_list = []
    
    loss_list = []
    for seq, space_data, label in tqdm(train_loader):
        seq = paddle.reshape(seq, (-1,4,size,size ))
        seq = paddle.cast(seq, dtype='float32')
        space_data = paddle.reshape(space_data, (-1,1,11,11 ))
        space_data = paddle.cast(space_data, dtype='float32')
        label = paddle.cast(label, dtype='float32')
        
        pred = model(seq, space_data)
        
        loss = criterion(pred.squeeze()/1000, label/1000)
        loss_list.append(loss)
        loss.backward()
        optimizer.step()
        #model.zero_grad()
        optimizer.clear_grad()
        pred_list.extend(pred.squeeze().cpu().detach().numpy())
        label_list.extend(label.squeeze().cpu().detach().numpy())
        del pred 
        del seq
        del space_data
        del label
    
    total_loss = paddle.mean(paddle.to_tensor(loss_list))
    model.eval()

    print(
        f'Epoch: {epoch} Loss: {total_loss}'
    )
    

  0%|          | 0/347 [00:00<?, ?it/s]/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/nn/layer/norm.py:654: UserWarning: When training, we now always track global mean and variance.
  "When training, we now always track global mean and variance.")
  0%|          | 0/347 [00:00<?, ?it/s]

Epoch: 0 Loss: Tensor(shape=[1], dtype=float32, place=Place(gpu:0), stop_gradient=True,
       [0.31255785])


  0%|          | 0/347 [00:00<?, ?it/s]

Epoch: 1 Loss: Tensor(shape=[1], dtype=float32, place=Place(gpu:0), stop_gradient=True,
       [0.30441925])


  0%|          | 0/347 [00:00<?, ?it/s]

...

Epoch: 47 Loss: Tensor(shape=[1], dtype=float32, place=Place(gpu:0), stop_gradient=True,
       [0.16579750])


  0%|          | 0/347 [00:00<?, ?it/s]

Epoch: 48 Loss: Tensor(shape=[1], dtype=float32, place=Place(gpu:0), stop_gradient=True,
       [0.16577452])


100%|██████████| 347/347 [00:04<00:00, 78.64it/s] 

Epoch: 49 Loss: Tensor(shape=[1], dtype=float32, place=Place(gpu:0), stop_gradient=True,
       [0.16576201])


In [38]:
model.eval()
layer_state_dict = model.state_dict()
paddle.save(layer_state_dict, "CNNSP_2022.pdparams")